To clone this repository use:
    git clone Minhaj0/MultiLabel-Classification-For-Github-Issues

In [68]:
import json
import requests
drive_link = "https://githubissuesdata.s3.us-east-2.amazonaws.com/data.txt"
git_issues_data=[]
response = requests.get(drive_link)
if response.status_code == 200:
    git_issues_data = json.loads(response.text)
    print("Total Numbers of issues:", len(git_issues_data))
else:
    print("Failed to download the file")
read_file=open("data.txt",'r')
git_issues_data=read_file.read()

Total Numbers of issues: 36816


In [67]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
for i in range(len(git_issues_data)):
    text=''
    if 'body' in git_issues_data[i] and git_issues_data[i]['body'] is not None:
        git_issues_data[i]['body'] = ' '.join([word for word in git_issues_data[i]['body'].split() if word not in STOPWORDS])
    else:
        git_issues_data[i]['body'] = ''
print("Total Numbers of issues:",len(git_issues_data))

Total Numbers of issues: 36816


In [62]:
dataset_for_training_model=[]
all_labels=[]
for item in git_issues_data:
    labels=[]
    labels.extend([
        obj['name']
        for obj in item['labels']
    ])
    all_labels.extend([*labels])
    if len(labels) > 0:
        dataset_for_training_model.extend([{
        'id':item['id'],
        'body':item['body'],
        'url':item['url'],
        'title':item['title'],
        'language':item['language'],
        'labels':labels
    }])
print(len(dataset_for_training_model))

18561


In [63]:
import pandas as pd
import numpy as np
df = pd.DataFrame(dataset_for_training_model)
df['body'].replace('', np.nan, inplace=True)
df = df.drop(['id', 'url'], axis =1)
df.dropna(subset=['body'], inplace=True)
print(df.head())

                                                body  \
0  Bumps [Selenium.WebDriver.ChromeDriver](https:...   
1  Bumps [Microsoft.SourceLink.GitHub](https://gi...   
2  ### Describe feature Based available HTML elem...   
3  ### Describe feature Based available HTML elem...   
4  ### Describe feature Simply, upgrading current...   

                                               title language  \
0  Bump Selenium.WebDriver.ChromeDriver from 119....     .NET   
1  Bump Microsoft.SourceLink.GitHub from 1.1.1 to...     .NET   
2  [Feature] Page object generation for Vue.js files     .NET   
3    [Feature] Page object generation for HTML files     .NET   
4                    [Feature] Upgrade to Selenium 4     .NET   

                                         labels  
0                          [dependencies, .NET]  
1                          [dependencies, .NET]  
2  [enhancement, good first issue, help wanted]  
3  [enhancement, good first issue, help wanted]  
4              [enhanc

In [64]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
df['text'] = df['body'] + ' ' + df['title'] + ' ' + df['language']
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(df['text'])
y = df['labels'].str.get_dummies(sep=',')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
multi_target_forest = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
multi_target_forest.fit(X_train, y_train)
y_pred = multi_target_forest.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7483766233766234
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       1.00      0.50      0.67         2
           2       1.00      0.80      0.89         5
           3       0.50      1.00      0.67         1
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         1
          15       1.00      0.67      0.80         3
          16       0.00      

/Users/xyz/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xyz/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xyz/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
